In [4]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install --upgrade selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_reviews():
    service = Service(GeckoDriverManager().install())
    driver = webdriver.Firefox(service=service)
    wait = WebDriverWait(driver, 10)
    
    try:
        # Open in reviews mode directly
        url = "https://www.google.com/maps/place/Couvee+-+Seturan/reviews"
        driver.get(url)
        
        # Wait for reviews container
        reviews_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.fontBodyMedium')))
        
        # Sort by newest
        sort_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-value="Sort"]')))
        sort_button.click()
        newest_option = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[data-index="1"]')))
        newest_option.click()
        
        reviews = []
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        while True:
            # Scroll down
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(2)
            
            # Get reviews
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.jJc9Ad')
            for element in review_elements:
                if element.text and element.text not in [r['text'] for r in reviews]:
                    reviews.append({
                        'text': element.text,
                        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S')
                    })
            
            # Check scroll position
            new_height = driver.execute_script("return document.documentElement.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
            
        return reviews
            
    finally:
        driver.quit()